# PyTorch quickstart

> based on https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

In [1]:
import time

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchinfo import summary
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [3]:
batch_size = 128

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([128, 1, 28, 28])
Shape of y: torch.Size([128]) torch.int64


In [4]:
def get_device():
    if not torch.backends.mps.is_available():
        if not torch.backends.mps.is_built():
            print(
                "MPS not available because the current PyTorch install was not "
                "built with MPS enabled."
            )
        else:
            print(
                "MPS not available because the current MacOS version is not 12.3+ "
                "and/or you do not have an MPS-enabled device on this machine."
            )
        return "cpu"
    return "mps"


# Get cpu or gpu device for training.
device = get_device()
print(f"Using {device=}")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to(device)
summary(model)

Using device='mps'


Layer (type:depth-idx)                   Param #
NeuralNetwork                            --
├─Flatten: 1-1                           --
├─Sequential: 1-2                        --
│    └─Linear: 2-1                       401,920
│    └─ReLU: 2-2                         --
│    └─Linear: 2-3                       262,656
│    └─ReLU: 2-4                         --
│    └─Linear: 2-5                       5,130
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [7]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n"
    )

In [8]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    start_time = time.monotonic()
    train(train_dataloader, model, loss_fn, optimizer)
    end_time = time.monotonic()
    total_time = end_time - start_time
    print(f"time: {total_time:.2f}s")
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302984  [    0/60000]
loss: 2.293267  [12800/60000]
loss: 2.275424  [25600/60000]
loss: 2.276091  [38400/60000]
loss: 2.253738  [51200/60000]
time: 2.75s
Test Error: 
 Accuracy: 30.6%, Avg loss: 2.246284 

Epoch 2
-------------------------------
loss: 2.243009  [    0/60000]
loss: 2.225797  [12800/60000]
loss: 2.215055  [25600/60000]
loss: 2.216061  [38400/60000]
loss: 2.186934  [51200/60000]
time: 2.59s
Test Error: 
 Accuracy: 40.1%, Avg loss: 2.176550 

Epoch 3
-------------------------------
loss: 2.172128  [    0/60000]
loss: 2.144170  [12800/60000]
loss: 2.134263  [25600/60000]
loss: 2.131856  [38400/60000]
loss: 2.093019  [51200/60000]
time: 2.56s
Test Error: 
 Accuracy: 48.4%, Avg loss: 2.077129 

Epoch 4
-------------------------------
loss: 2.070018  [    0/60000]
loss: 2.027827  [12800/60000]
loss: 2.015166  [25600/60000]
loss: 2.004114  [38400/60000]
loss: 1.957776  [51200/60000]
time: 2.73s
Test Error: 
 Accuracy: 56.0%, Avg l

In [9]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x.to(device))
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
